In [1]:
import sys
sys.path.append('c:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('c:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('c:\\software_git_repos\\greenpol')
sys.path.append('c:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('c:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
from glob import glob
import time
import cofe_util as cu
import Tkinter,tkFileDialog
root = Tkinter.Tk()
from demod import datparsing
%pylab

import realtime_gp as rt

C:\Users\peter\AppData\Local\Continuum\anaconda3\envs\py27\lib\site-packages\pyfits\__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\peter\AppData\Local\Continuum\anaconda3\envs\py27\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [3]:
def bin_to_az_el_patch(indata,azres=0.5,elres=0.5,chan='ch4',cmode='Q',revlimits=[0,2**24]):
    """
    straight binning code for data that's been associated with H5 pointing
    assume combined_dict came from combine_cofe_h5_pointing, that az and el are in degrees
    Try to make this version only where we have data in az and el (for small patches)
    start by assuming we don't cross az=0, fix this case if this is useful
    """

    chandata=indata['sci_data'][chan][cmode][np.logical_and(indata['sci_data']['rev'] > revlimits[0],indata['sci_data']['rev']< revlimits[1])]
    eldata=indata['el'][np.logical_and(indata['sci_data']['rev'] > revlimits[0],indata['sci_data']['rev']< revlimits[1])]
    azdata=indata['az'][np.logical_and(indata['sci_data']['rev'] > revlimits[0],indata['sci_data']['rev']< revlimits[1])]
    
    elhalfbin=elres/2.
    azhalfbin=azres/2.
    azmin=np.min(azdata)
    azmax=np.max(azdata)
    elmin=np.min(eldata)
    elmax=np.max(eldata)
    nazbins=int((azmax-azmin)/azres)
    nelbins=int((elmax-elmin)/elres)
    azlist=arange(nazbins)*azres+azmin
    ellist=arange(nelbins)*elres+elmin
    
    # make empty output map
    outmap=np.zeros([nazbins,nelbins],dtype=np.float64) 
    for x,azi in enumerate(azlist):
        for y,eli in enumerate(ellist):
            outmap[x,y]=np.mean(chandata[np.logical_and(abs(eldata-eli) < elhalfbin,abs(azdata-azi) < azhalfbin)])
    mapoffset=np.nanmin(outmap)
    outmap=np.nan_to_num(outmap-mapoffset)
    outmapdict={}
    x,y=np.meshgrid(azlist,ellist)
    outmapdict['az']=x
    outmapdict['el']=y
    outmapdict['map']=transpose(outmap)
    return outmapdict

In [4]:
pointing_list=list(tkFileDialog.askopenfilenames(initialdir='C://Greenland/test_data_from_greenpol_1/pointing_data',parent=root,title='Choose files'))

In [5]:
data_list=list(tkFileDialog.askopenfilenames(initialdir='C://Greenland/test_data_from_greenpol_1/data',parent=root,title='Choose files'))

In [6]:
pp=rt.get_h5_pointing(pointing_list)

In [17]:
reload(datparsing)

<module 'demod.datparsing' from 'c:\software_git_repos\cofe-python-analysis-tools\utils_zonca\demod\datparsing.py'>

In [18]:
dd=rt.get_demodulated_data_from_list(data_list)

1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16
1000
16


In [19]:
cdata=rt.combine_cofe_h5_pointing(dd,pp,outfile='cdata.pkl')

In [37]:
figure()
plot(cdata['az'],cdata['sci_data']['ch4']['T'])

In [35]:
testh2t=bin_to_az_el_patch(cdata,chan='ch4',cmode='T',azres=.2,elres=0.2)

C:\Users\peter\AppData\Local\Continuum\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Warning: converting a masked element to nan.


In [36]:
figure()
pcolormesh(testh2t['az'],testh2t['el'],testh2t['map'])
title('Horn 2 T')
colorbar()